## 준비

In [1]:
import math
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input

In [2]:
# Jupyter notebook 경고 메시지 숨기기
from warnings import filterwarnings
filterwarnings(action="ignore")

## 모델 정의

In [3]:
TRAIN_DATA_DIR = "drive/MyDrive/data/train/"
VAL_DATA_DIR = "drive/MyDrive/data/val/"
TRAIN_SAMPLES = 500
VAL_SAMPLES = 500
NUM_CLASSES = 2
IMG_WIDTH, IMG_HEIGHT = 224, 224
BATCH_SIZE = 64

In [4]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [5]:
train_generator = train_datagen.flow_from_directory(
    TRAIN_DATA_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    shuffle=True,
    class_mode="categorical"
)

Found 500 images belonging to 2 classes.


In [6]:
val_generator = val_datagen.flow_from_directory(
    VAL_DATA_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    shuffle=False,
    class_mode="categorical"
)

Found 500 images belonging to 2 classes.


In [7]:
def model_maker():
    base_model = MobileNetV2(include_top=False, input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))

    for layer in base_model.layers:
        layer.trainable = False
    
    input = keras.layers.Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    custom_model = base_model(input)
    custom_model = keras.layers.GlobalAveragePooling2D()(custom_model)
    custom_model = keras.layers.Dense(64, activation="relu")(custom_model)
    custom_model = keras.layers.Dropout(0.5)(custom_model)
    predictions = keras.layers.Dense(NUM_CLASSES, activation="softmax")(custom_model)
    return keras.models.Model(inputs=input, outputs=predictions)

In [ ]:
model = model_maker()
model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(lr=1e-3), metrics=["acc"])
num_steps = math.ceil(float(TRAIN_SAMPLES) / BATCH_SIZE)

In [9]:
history = model.fit_generator(
    generator=train_generator,
    steps_per_epoch=num_steps,
    epochs=10,
    validation_data=val_generator,
    validation_steps=num_steps
)

Epoch 1/10
8/8 [==============================] - 293s 39s/step - loss: 0.4926 - acc: 0.7720 - val_loss: 0.1140 - val_acc: 0.9600
Epoch 2/10
8/8 [==============================] - 36s 5s/step - loss: 0.1494 - acc: 0.9420 - val_loss: 0.0556 - val_acc: 0.9740
Epoch 3/10
8/8 [==============================] - 36s 5s/step - loss: 0.1472 - acc: 0.9500 - val_loss: 0.0486 - val_acc: 0.9840
Epoch 4/10
8/8 [==============================] - 26s 3s/step - loss: 0.0863 - acc: 0.9680 - val_loss: 0.0456 - val_acc: 0.9820
Epoch 5/10
8/8 [==============================] - 27s 3s/step - loss: 0.0653 - acc: 0.9800 - val_loss: 0.0526 - val_acc: 0.9840
Epoch 6/10
8/8 [==============================] - 27s 3s/step - loss: 0.0741 - acc: 0.9740 - val_loss: 0.0451 - val_acc: 0.9820
Epoch 7/10
8/8 [==============================] - 26s 3s/step - loss: 0.0748 - acc: 0.9680 - val_loss: 0.0588 - val_acc: 0.9820
Epoch 8/10
8/8 [==============================] - 27s 4s/step - loss: 0.0628 - acc: 0.9780 - val_loss:

## 샘플 이미지 로딩 후 결과 보기

In [10]:
def preprocess_img(img_path):
    img = keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
    img_array = keras.preprocessing.image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = expanded_img_array / 255.0
    return preprocessed_img

In [11]:
dog_img = preprocess_img("drive/MyDrive/data/sample-images/dog.jpg")
model.predict(dog_img)

array([[0.00150615, 0.9984938 ]], dtype=float32)

In [12]:
cat_img = preprocess_img("drive/MyDrive/data/sample-images/cat.jpg")
model.predict(cat_img)

array([[9.9996734e-01, 3.2649947e-05]], dtype=float32)